<a href="https://colab.research.google.com/github/Yongshilin/GIS/blob/main/gisdata_gpx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install geopandas shapely gpxpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
import os
import gpxpy
import geopandas as gpd
from shapely.geometry import LineString
import zipfile
import pandas as pd

def gpx_to_geodataframe(gpx_file):
    with open(gpx_file, 'r') as file:
        gpx = gpxpy.parse(file)

    lines = []  # List to hold LineString geometries
    for track in gpx.tracks:
        all_coords = []  # List to hold all coordinates for this track
        for segment in track.segments:
            # Create LineString from segment points
            coords = [(point.longitude, point.latitude) for point in segment.points]
            all_coords.extend(coords)  # Add segment coordinates to the track list

        if all_coords:  # Ensure there are coordinates to create a LineString
            line = LineString(all_coords)
            lines.append({
                'geometry': line,
                'elevation': [point.elevation for point in segment.points],  # Optional: Add elevation
                'time': [point.time for point in segment.points]  # Optional: Add time
            })

    return gpd.GeoDataFrame(lines, crs="EPSG:4326")  # Set the CRS for WGS84

def merge_gpx_files(gpx_files):
    all_lines = []
    for gpx_file in gpx_files:
        gdf = gpx_to_geodataframe(gpx_file)
        all_lines.append(gdf)

    return gpd.GeoDataFrame(pd.concat(all_lines, ignore_index=True))

def save_shapefile_as_zip(gdf, shapefile_name, output_folder):
    # Construct the shapefile path
    shapefile_path = os.path.join(output_folder, f"{shapefile_name}.shp")

    # Save as shapefile
    gdf.to_file(shapefile_path, driver="ESRI Shapefile")

    # Zip the shapefile components
    shapefile_zip = f"{shapefile_path}.zip"
    with zipfile.ZipFile(shapefile_zip, 'w') as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                if file.startswith(shapefile_name) and file.endswith(('.shp', '.shx', '.dbf', '.prj')):
                    zipf.write(os.path.join(root, file), arcname=file)

In [ ]:
# List of GPX files to merge
gpx_files = ['/content/Betty Steflik Preserve Trail.gpx',
             '/content/Bulow Woods Hiking Trail.gpx',
             '/content/Coastal Strand Trail.gpx',
             '/content/Fairchild Oak Trail.gpx',
             '/content/Graham Swamp Mountain Bike Trail.gpx',
             '/content/Graham Swamp Trail.gpx',
             '/content/Intracoastal Waterway Trail.gpx',
             '/content/Jungle Hut Loop Trail.gpx',
             '/content/Lehigh Rail Trail.gpx',
             '/content/Ormond Scenic Loop and Trail.gpx',
             '/content/Sabal Palm and Palmetto Loop.gpx']  # Add your GPX file paths here
output_folder = "/content/output_folder"  # Specify your output directory
shapefile_name = "flagler_11trails"  # Specify your shapefile name

# Merge GPX files and save as a shapefile zip
merged_gdf = merge_gpx_files(gpx_files)
save_shapefile_as_zip(merged_gdf, shapefile_name, output_folder)


/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Value '[2.37, 2.48, 2.55, 2.0, 1.0, 0.99, 0.31, -0.3, -1.0, 0.0, -0.17, -0.51, 0.36, 0.54, 0.54, 0.36, -0.51, -0.17, 0.0, -1.0, -0.3, 0.31, 0.99, 1.0, 0.9, 0.69, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 0.35, 0.56, 0.81, 0.59, 0.76, 0.72, 0.51, 0.18, -0.02, -0.05, 0.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.31, -0.84, -1.0, -0.12, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, -0.68, -0.83, -0.85, -1.0, -0.67, -0.74, -1.0, -0.62, -0.7, -0.59, -1.0, -0.82, -0.95, -1.0, -0.95, -0.69, -1.0, -1.0, -1.0, -0.96, 0.0, 0.52, 0.5, 0.5, 0.52, 0.0, -0.96, -1.0, -1.0, -1.0, -0.69, -0.95, -1.0, -0.95, -0.82, -1.0, -0.59, -0.7, -0.62, -1.0, -0.74, -0.67, -1.0, -0.85, -0.83, -0.68, 0.0, 0.0, 0.0, 0.0, 0.0, -0.12, -1.0, -0.84, -0.31, 0.0, 1.0, 1.0, 2.14, 3.0, 3.0, 3.0, 3.0, 2.48, 2.37]' of field elevation has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


In [ ]:
#Verify Geometry Types
#If the output shows Point instead of LineString, then the problem lies in how the geometries are being created or merged.
print(merged_gdf.geometry.geom_type)


0     LineString
1     LineString
2     LineString
3     LineString
4     LineString
5     LineString
6     LineString
7     LineString
8     LineString
9     LineString
10    LineString
dtype: object


In [ ]:
#Check for Valid Geometries
print(merged_gdf.geometry.is_valid)


0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
dtype: bool


In [ ]:
#If any of the geometries are invalid, you can try to fix them using the buffer(0) method, which can help to correct minor issues:
merged_gdf['geometry'] = merged_gdf.geometry.buffer(0)


In [ ]:
# Download shpefile zip file to the computer
from google.colab import files
files.download('/content/output_folder/flagler_11trails.shp.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>